In [1]:
import os

from QHyper.problems.algorithms.solver_utils import WorkflowSchedulingSolverDecorator, \
    DecomposedWorkflowSchedulingSolver
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm, add_entry_and_exit_tasks, \
    SeriesParallelSplit
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingOneHot
from QHyper.solvers import solver_from_config
from QHyper.solvers.gurobi import Gurobi
from reports.report import ExecutionReport, Solution

In [2]:
def get_solver(tasks_file: str, machines_file: str, deadline: int):
    solver_config = {
        "problem": {
            "type": "workflow_scheduling",
            "encoding": "one-hot",
            "tasks_file": tasks_file,
            "machines_file": machines_file,
            "deadline": deadline,
        },
        "solver": {
            "type": "gurobi",
        }
    }

    return WorkflowSchedulingSolverDecorator(solver_from_config(solver_config))


def get_filename(tasks_file: str):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}.json"


def run_tests(test_data, save_dir):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        gurobi = get_solver(tasks_file, machines_file, deadline)
        schedule = gurobi.solve()
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solver="Gurobi",
            solution=Solution.from_workflow_schedule(schedule)
        )
        report.write_json(os.path.join(save_dir, get_filename(tasks_file)))

In [5]:
tasks_files = [
    # "workflows_data/workflows/srasearch_22_tasks.json",
    # "workflows_data/workflows/1000genome_156_tasks.json",
    "workflows_data/workflows/1000genome_492_tasks.json",
    # "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    # "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json"
]

deadlines = [
    # 3000,
    # 2000,
    2000,
    # 5000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [6]:
run_tests(test_data, "reports/linear_machines/3000/after_changes/no_division")

Executing test for workflows_data/workflows/1000genome_492_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)
CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


Consider calling update less frequently.

Optimize a model with 4860 rows, 2460 columns and 67140 nonzeros
Model fingerprint: 0x9ee538a9
Variable types: 0 continuous, 2460 integer (2460 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 1564859.6620
Presolve removed 4518 rows and 2300 columns
Presolve time: 2.22s
Presolved: 342 rows, 160 columns, 3034 nonzeros
Found heuristic solution: objective 844638.34237
Variable types: 0 continuous, 160 integer (160 binary)

Root relaxation: objective 8.145277e+05, 68 iterations, 0.00 seconds (0.00 work uni

In [7]:
def get_decomposed_solver(tasks_file: str, machines_file: str, deadline: int, max_graph_size: int):
    workflow = Workflow(tasks_file, machines_file, deadline)
    division = SeriesParallelSplit().decompose(workflow, max_graph_size)
    problems = map(lambda w: WorkflowSchedulingOneHot(w), division.workflows)
    solvers = map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)
    return DecomposedWorkflowSchedulingSolver(list(solvers), division)


def get_filename(tasks_file: str, max_graph_size: int):
    workflow_name, _ = (tasks_file.split(sep="/")[-1]).split(sep=".")
    return f"gurobi_{workflow_name}_mfs_{max_graph_size}.json"


def run_tests_division(test_data, save_dir, max_graph_size):
    for tasks_file, machines_file, deadline in test_data:
        print(f"Executing test for {tasks_file}...")
        solver = get_decomposed_solver(tasks_file, machines_file, deadline, max_graph_size)
        schedule = solver.solve()
        report = ExecutionReport(
            workflow_file=tasks_file,
            machines_file=machines_file,
            deadline=deadline,
            solver="Gurobi",
            solution=Solution.from_workflow_schedule(schedule),
            max_graph_size=max_graph_size
        )
        report.write_json(os.path.join(save_dir, get_filename(tasks_file, max_graph_size)))

In [21]:
tasks_files = [
    # "workflows_data/workflows/srasearch_22_tasks.json",
    # "workflows_data/workflows/1000genome_156_tasks.json",
    "workflows_data/workflows/1000genome_492_tasks.json",
    # "workflows_data/workflows/1000genome_902_tasks.json"
]

machines_files = [
    # "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json",
    "workflows_data/machines/linear.json",
    # "workflows_data/machines/linear.json"
]

deadlines = [
    # 3000,
    # 2000,
    2000,
    # 3000
]

test_data = zip(tasks_files, machines_files, deadlines)

In [22]:
run_tests_division(test_data, "reports/linear_machines/3000/after_changes/division", 8)

Executing test for workflows_data/workflows/1000genome_492_tasks.json...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 15 columns and 20 nonzeros
Model fingerprint: 0x2893d395
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 104.3223552
Presolve removed 4 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 104.322 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.043223552000e+02, best bound 1.043223552000e+

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

